# F1 ENGINE PERFORMANCE OVER TIME

> Members:
* Anna Czarnocka
* Gabor Swistak
* Ansh Varma
* Camilo Ulloa

## WEB SCRAPING SECTION

> In this first section, we will focus on creating web scrapping models for obtaining and creating the data for out analysis

> We will start by importing all the neccesary libraries

In [245]:
import requests
from bs4 import BeautifulSoup
from csv import writer
import pandas as pd
import numpy as np
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from contextlib import contextmanager


## F1 Teams Table

> In this section we will use web scrapping for obtaining the data of all the teams that raced from 2014 to 2023. 

> Moreover, our range of years was based on the fact that in 2014, there were major changes in the regulations of F1 cars engines. For instance, they change their cylinder from V8 to V6 as well as the horsepower and more details regarding weight and fuel. Therefore, we wil focus our study after the implementation of this regulations.

In [3]:
class F1TeamScraper:
    def __init__(self, base_url):
        self.base_url = base_url
        self.teams_data = []

    def scrape_teams_for_year(self, year):
        url = self.base_url.format(year) # We introduce the format of "year" so later we can loop through the range of years of interest
        page = requests.get(url)
        try:
            soup = BeautifulSoup(page.content, 'html.parser')
        except page.status_code != 200: # If the status code of the page is != 200 it means Beautiful Soup could read correctly the web page and access its information. 
            pass
        table = soup.find('table') # We will look for the element "table" in the HTML code
        rows = table.find_all('tr')[1:] # Then we will find the father element "tr"
        teams = [row.find_all('td')[2].text.strip() for row in rows] # Finally we will extract the information from the son "td" where is the information regarding the teams
        self.teams_data.extend([(year, team) for team in teams])

    def scrape_range_of_years(self, start_year, end_year):
        for year in range(start_year, end_year + 1): # We loop throught our years of interest 2014-2023
            self.scrape_teams_for_year(year)

    def get_teams_data(self):
        return pd.DataFrame(self.teams_data, columns=['Year', 'Teams']) # Finally it returns a Data Frame with the data

# Initiate the scraper with the base URL
scraper = F1TeamScraper("https://www.formula1.com/en/results.html/{}/team.html") 

# Scrape the team data for the range of years 2014-2023
scraper.scrape_range_of_years(2014, 2023)

# Retrieve the scraped data as a DataFrame
f1_teams_df = scraper.get_teams_data()

f1_teams_df.head()


,Year,Teams
0,2014,Mercedes
1,2014,Red Bull Racing Renault
2,2014,Williams Mercedes
3,2014,Ferrari
4,2014,McLaren Mercedes


In [4]:
f1_teams_df.shape

(103, 2)

In [5]:
f1_teams_df.to_csv("/Users/patricioulloa/UNI/UBC/Economic~DS/project/f1_teams.csv") # We save this first table

In [4]:
f1_teams_df = pd.read_csv("/Users/patricioulloa/UNI/UBC/Economic~DS/project/f1_teams.csv")

> Once we got the data for all the teams from 2014 to 2023, we will use this information for obtaining more information of each team regarding:
* The engine they use in different years
* The number of points
* The pole positions
* The number of wins
* The number of fastest laps
* The world champion position they had at the end of each season

> However, before getting to the extraction of this additional data, we will have to modify the **f1_teams_df** dataframe. Specifically, we will have to create a new column where we will only get the team name without the engine of sponsor name. For example, in 2014, Red Bull used the engine of Renault, therefore their name is Red Bull Racing Renault. Nevertheless, we will only be interested in getting the name Red Bull Racing without the engine Renault.

> This will provide us with the key words for extracting the additional data of each team across the years of analysis.

In [8]:
def extract_and_format_team_names(df):
    # Extracting team names and formatting them: lower case and replace spaces with hyphens
    df['Team Name'] = df['Teams'].apply(
        lambda x: ' '.join(x.split()[:3]).lower().replace(' ', '-') if 'Racing' in x else x.split()[0].lower()
    )
    return df

f1_teams_formatted_df = extract_and_format_team_names(f1_teams_df)
f1_teams_formatted_df = f1_teams_formatted_df.drop("Unnamed: 0", axis=1)
f1_teams_formatted_df.head()


,Year,Teams,Team Name
0,2014,Mercedes,mercedes
1,2014,Red Bull Racing Renault,red-bull-racing
2,2014,Williams Mercedes,williams
3,2014,Ferrari,ferrari
4,2014,McLaren Mercedes,mclaren


> Since we are using the names of teams for the official F1 website as a key to obtain the rest of the data from other website, the name of the teams have some changes. Therefore, we needed to make some adjustments so the name of the teams were the same as in the Motorsport website. In this way, the web scrapping model will identify correctly the names and extract all the data. 

In [9]:
f1_teams_formatted_df['Team Name'] = f1_teams_formatted_df['Team Name'].replace('ferrari', 'ferrari-2')
f1_teams_formatted_df['Team Name'] = f1_teams_formatted_df['Team Name'].replace('lotus', 'lotus-f1')
f1_teams_formatted_df['Team Name'] = f1_teams_formatted_df['Team Name'].replace('racing-point-bwt', 'bwt-racing-point-f1-team')
f1_teams_formatted_df['Team Name'] = f1_teams_formatted_df['Team Name'].replace('alphatauri', 'scuderia-alphatauri-honda')
f1_teams_formatted_df['Team Name'] = f1_teams_formatted_df['Team Name'].replace('alfa-romeo-racing', 'alfa-romeo')
f1_teams_formatted_df['Team Name'] = f1_teams_formatted_df['Team Name'].replace('alfa', 'alfa-romeo')
f1_teams_formatted_df['Team Name'] = f1_teams_formatted_df['Team Name'].replace('aston', 'aston-martin')
f1_teams_formatted_df['Team Name'] = f1_teams_formatted_df['Team Name'].replace('force', 'force-india')
f1_teams_formatted_df.head()

,Year,Teams,Team Name
0,2014,Mercedes,mercedes
1,2014,Red Bull Racing Renault,red-bull-racing
2,2014,Williams Mercedes,williams
3,2014,Ferrari,ferrari-2
4,2014,McLaren Mercedes,mclaren


In [59]:
f1_teams_formatted_df.to_csv("/Users/patricioulloa/UNI/UBC/Economic~DS/project/f1_teams_formatted.csv") 

## Data Table

> Now that we have the **f1_teams_formatted_df** cleaned table, we can extract the data regarding:
* The number of points
* The pole positions
* The number of wins
* The number of fastest laps
* The world champion position they had at the end of each season

> For this section, we will extract the data from the web page Motor Sport Stats. Since this web page is dynamic and uses Java Script, we will use the library **Selenium** for obtaining the data and then parse it with Beautiful Soup. Morever, since Selenium act as a bot, it offers the possibility of moving around the web page and interact with it. Therefore, it will be very usefull for future web scraping

In [29]:
class F1TeamDataScraper:
    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.team_years = self.extract_teams_and_years()
        '''
        Selenium uses a Web Driver that is a collection of open source APIs which are used to automate the testing of a web application.
        Therefore, we need to download the Web Driver that is compatible with our current version of Google Chrome and insert it's location.
        '''
        self.driver = webdriver.Chrome("/usr/local/bin/chromedriver") 

    def extract_teams_and_years(self):
        '''
        With this function, we aim to extract the unique teams and the years that each unique team has raced for our time of analysis. 
        The objective is to create a list that we can loop throught later for obtaining the data of interest.
        '''
        team_years_list = []
        for team in self.dataframe['Team Name'].unique():
            years = self.dataframe[self.dataframe['Team Name'] == team]['Year'].unique()
            team_years_list.append((team, years))
        return team_years_list

    def scrape_data_for_team(self, team, years):
        '''
        This function is the web scraping model for obtaining our desire data
        '''
        self.driver.get(f"https://www.motorsportstats.com/team/{team}/summary/series/fia-formula-one-world-championship")
        time.sleep(5) # We add this time sleep so we can give time to the dynamic web page to charge and show us information
        page_source = self.driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        time.sleep(2)  
        
        summary_table = soup.find("table") 
        if summary_table is None: # We use this conditional to prove that there exist data for the team we are scraping its data
            print(f"No data found for team: {team}")
            return pd.DataFrame()  # Return an empty DataFrame if no table found

        rows = summary_table.find_all("tr") # Again "tr" is the father in the HTML code

        # Extracting table headers
        headers = [header.get_text().strip() for header in rows[0].find_all("th") if header.get_text().strip() != "Drivers"]
        
        data = []
        for row in rows[1:]:  # Skip the header row
            cols = row.find_all("td")
            year_text = cols[0].get_text().strip() # With this we are obtaining the years from the table of the web page
            if year_text.isdigit():
                year = int(year_text)
                if year in years:
                    '''
                    With this conditional we are checking if the year we got previously, exist in the data we have as reference from "f1_teams_formatted_df".
                    If it happens to exist, then we extract all the data of the column regarding that year
                    '''
                    filtered_cols = [col.get_text().strip() for col in cols if col != cols[1]]
                    if len(filtered_cols) == len(headers):
                        data.append(filtered_cols)

        # Create a DataFrame from the extracted data
        return pd.DataFrame(data, columns=headers)

    def run_scraping(self):
        '''
        This function runs the web scraping model
        Since we have many teams with many years, we are going to store the data in a dictionary a later trasnform it to a dataframe.
        '''
        team_data = {}
        for team, years in self.team_years:
            df = self.scrape_data_for_team(team, years)
            team_data[team] = df
            print(f"Scraped data for {team}")
        self.driver.quit()
        return team_data


file_path = '/Users/patricioulloa/UNI/UBC/Economic~DS/project/f1_teams_formatted.csv'
f1_teams_data_formatted_df = pd.read_csv(file_path)
scraper = F1TeamDataScraper(f1_teams_data_formatted_df) # We initiate the scraper model
all_team_data = scraper.run_scraping()


<ipython-input-29-dab8ad1f01aa>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome("/usr/local/bin/chromedriver")


Scraped data for mercedes
Scraped data for red-bull-racing
Scraped data for williams
Scraped data for ferrari-2
Scraped data for mclaren
Scraped data for force-india
No data found for team: str
Scraped data for str
Scraped data for lotus-f1
Scraped data for marussia
Scraped data for sauber
Scraped data for caterham
No data found for team: toro
Scraped data for toro
Scraped data for haas
No data found for team: renault
Scraped data for renault
No data found for team: mrt
Scraped data for mrt
No data found for team: scuderia
Scraped data for scuderia
Scraped data for bwt-racing-point-f1-team
Scraped data for alfa-romeo
Scraped data for scuderia-alphatauri-honda
Scraped data for alpine
Scraped data for aston-martin


In [33]:
# We convert the dictionary into a dataframe by concantenating their values and keys
data_table = pd.concat(all_team_data.values(), keys=all_team_data.keys()).reset_index(level=1, drop=True).reset_index()
data_table.rename(columns={'index': 'Team'}, inplace=True)
data_table.to_csv("/Users/patricioulloa/UNI/UBC/Economic~DS/project/data_table.csv")
data_table

,Team,Year,ST,W,PD,PP,FL,BF,BG,PTS,WC
0,mercedes,2023,20,0,7,1,5,2,1,382,2nd
1,mercedes,2022,22,1,17,2,6,1,1,515,3rd
2,mercedes,2021,22,9,28,9,10,1,1,613.5,1st
3,mercedes,2020,17,13,25,15,9,1,1,573,1st
4,mercedes,2019,21,15,32,10,9,1,1,739,1st
...,...,...,...,...,...,...,...,...,...,...,...
85,alpine,2022,22,0,0,0,0,4,2,173,4th
86,alpine,2021,22,1,2,0,0,1,3,155,5th
87,aston-martin,2023,20,0,8,0,1,2,2,261,5th
88,aston-martin,2022,22,0,0,0,0,6,5,55,7th


## Engine Table

> Now, we would extract the data regarding each team engine, the Grid Position and the final position of the driver at the end of each race, across our years of analysis (2014-2023)

> Once we got this data, we can merge this **engine table** with **data_table** to have our final table for the EDA

In [211]:
@contextmanager
def ChromeDriverContext(driver_path):
    '''
    The objective of this function is to handle errors if the web driver couldn't acces the web page or some error occured in the process
    '''
    driver = webdriver.Chrome(driver_path)
    try:
        yield driver
    except Exception as e:
        print(f"An error occurred: {e}")
        raise
    finally:
        driver.quit()


class F1TeamDataScraper:
    def __init__(self, driver_path):
        self.driver_path = driver_path

    def scrape_data_for_team(self, team):
        with ChromeDriverContext(self.driver_path) as driver:
            try:
                base_url = f"https://www.motorsportstats.com/team/{team}/stats/series/fia-formula-one-world-championship/starts"
                driver.get(base_url)
                
                # Wait for page to load
                time.sleep(5)

                '''
                Since there are teams in our data that the Motorsports web page doesn't have data of, we use the following code as a key to 
                identify if there exist a table with data inside of it. If it doesn't exist, then it means that the web page doesn't have 
                data for that teams and it returns an empty dataframe for that specific team
                '''
                td_elements = driver.find_elements(By.CSS_SELECTOR, "table.styled__Table-sc-2ks33b-2 tbody[role='rowgroup'] tr[role='row'] td[role='cell']")
                if not td_elements:
                    print(f"No data found for team: {team}")
                    return pd.DataFrame()
                
                '''
                In this part we aim to search for the dropdown menu in the web page which contains different options for the visualization of the
                data in the table. We will select the maximun option which is visualizing 100 elements at the same time. Out objctive it to
                reduce the amount of pages the web scraping model has to go throught to gather all the neccesary data for our time of analysis.
                For instance, it will be easier if by displaying the 100 elements we are left only with 4 pages of data of the table, than if
                we display only 20 elements, which in that case, we will have more than 10 pages.
                '''
                try:
                    dropdown = driver.find_element(By.ID, "itemsAtPage")
                    dropdown.click()
                    driver.find_element(By.XPATH, "//option[. = '100']").click()
                    time.sleep(6)
                except NoSuchElementException:
                    print(f"No data found for team: {team}")
                    return pd.DataFrame()
                
                '''
                There are teams like Ferrari for which the web page has data from 1950 till 2023. Therefore, instead of looking for the data of
                interest from the first page of data, it wil be faster if we directly go to the last page of data and start extracting the data
                backwards from 2023 till we reach 2014. That is precisely what the following code does
                '''
                try:
                    to_last_page_button = driver.find_element(By.CLASS_NAME, "styled__ToLastPage-sc-zuhhtt-5")
                    to_last_page_button.click()
                    time.sleep(10)
                except NoSuchElementException:
                    print(f"Last page button not found for team: {team}")
                    return pd.DataFrame()

                # Initialize a variable to store the extracted data
                data = []

                # Loop to navigate pages and extract data
                year_reached = False
                earliest_year_found = None
                min_year_threshold = 2014
                while not year_reached:
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    
                    # Extract data for rows with years 2014 to 2023
                    rows = soup.find_all('tr')
                    years_on_page = []
                    for row in rows:
                        cols = row.find_all('td')
                        if cols and len(cols) > 7: # Ensure there are enough columns
                            year = int(cols[1].text.strip())
                            years_on_page.append(year)
                            if year >= min_year_threshold:
                                # Extract data and append to the list
                                data.append({
                                    "SEQ": cols[0].text.strip(),
                                    "YEAR": year,
                                    "EVENT": cols[2].text.strip(),
                                    "DRIVER": cols[3].text.strip(),
                                    "CAR": cols[4].text.strip(),
                                    "ENGINE": cols[5].text.strip(),
                                    "GRID POS": cols[6].text.strip(),
                                    "RACE": cols[7].text.strip(),
                                })
                    
                    '''
                    There are teams for which the earliest data doesn't get to 2014. Therefore, we created the following conditional in a way
                    that it checks if the year for which it is extracting the data is less than the previous year, if this is true, then it 
                    continues to scrape the data. However if that year is less than 2014 then the scraping model stops. For instance, for the
                    team Haas, there is only data till 2016, so the web scrapping model will extract all the data till 2016 even if 2016 > 2014
                    '''
                    if years_on_page:
                        current_earliest_year = min(years_on_page)
                        if earliest_year_found is None or current_earliest_year < earliest_year_found:
                            earliest_year_found = current_earliest_year
                        else:
                            year_reached = True

                    if year_reached:
                        break
                    
                    '''
                    In this part, we locate in the HTML code the element for the button that get us to the previous page and we click it so we
                    can extract the rest of the data until we arrive to the year 2014
                    '''
                    prev_page_buttons = driver.find_elements(By.CLASS_NAME, "styled__PreviousPage-sc-zuhhtt-2")
                    if not prev_page_buttons:
                        break  # If no previous page button, exit the loop

                    # Click the previous page button
                    prev_page_buttons[0].click()
                    time.sleep(10)
                    

            finally:
                # Close the WebDriver
                driver.quit()
            
            # Return the data as a pandas DataFrame
            return pd.DataFrame(data)


scraper = F1TeamDataScraper('/usr/local/bin/chromedriver')
all_team_data = pd.DataFrame()

unique_teams = f1_teams_data_formatted_df['Team Name'].unique() # We extract the unique teams in the data so we don't have duplicated data
for team in unique_teams:
    team_data = scraper.scrape_data_for_team(team)
    if not team_data.empty:
        team_data["Team Name"] = team
        all_team_data = pd.concat([all_team_data, team_data], ignore_index=True)

print(all_team_data)


<ipython-input-211-d73a1697d6e0>:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


No data found for team: str
No data found for team: toro
No data found for team: renault
No data found for team: mrt
No data found for team: scuderia
      SEQ  YEAR                     EVENT            DRIVER  \
0     601  2023      Las Vegas Grand Prix    George Russell   
1     501  2021        Russian Grand Prix   Valtteri Bottas   
2     502  2021        Turkish Grand Prix   Valtteri Bottas   
3     503  2021        Turkish Grand Prix    Lewis Hamilton   
4     504  2021  United States Grand Prix    Lewis Hamilton   
...   ...   ...                       ...               ...   
4303   96  2022      São Paulo Grand Prix  Sebastian Vettel   
4304   97  2022      Abu Dhabi Grand Prix      Lance Stroll   
4305   98  2022      Abu Dhabi Grand Prix  Sebastian Vettel   
4306   99  2023        Bahrain Grand Prix   Fernando Alonso   
4307  100  2023        Bahrain Grand Prix      Lance Stroll   

                                CAR                         ENGINE GRID POS  \
0     Mercedes

In [219]:
all_team_data.to_csv("/Users/patricioulloa/UNI/UBC/Economic~DS/project/engine_data.csv") # We save the data
engine_data = pd.read_csv("/Users/patricioulloa/UNI/UBC/Economic~DS/project/engine_data.csv")
engine_data = engine_data.drop("Unnamed: 0", axis=1)
engine_data

,SEQ,YEAR,EVENT,DRIVER,CAR,ENGINE,GRID POS,RACE,Team Name
0,601,2023,Las Vegas Grand Prix,George Russell,Mercedes F1 W14 E Performance,Mercedes F1 M14 E Performance,3,8,mercedes
1,501,2021,Russian Grand Prix,Valtteri Bottas,Mercedes F1 W12 E Performance,Mercedes F1 M12 E Performance,16,5,mercedes
2,502,2021,Turkish Grand Prix,Valtteri Bottas,Mercedes F1 W12 E Performance,Mercedes F1 M12 E Performance,1,1,mercedes
3,503,2021,Turkish Grand Prix,Lewis Hamilton,Mercedes F1 W12 E Performance,Mercedes F1 M12 E Performance,11,5,mercedes
4,504,2021,United States Grand Prix,Lewis Hamilton,Mercedes F1 W12 E Performance,Mercedes F1 M12 E Performance,2,2,mercedes
...,...,...,...,...,...,...,...,...,...
4303,96,2022,São Paulo Grand Prix,Sebastian Vettel,NaN,NaN,9,11,aston-martin
4304,97,2022,Abu Dhabi Grand Prix,Lance Stroll,Aston Martin AMR22,Mercedes F1 M13 E Performance,14,8,aston-martin
4305,98,2022,Abu Dhabi Grand Prix,Sebastian Vettel,Aston Martin AMR22,Mercedes F1 M13 E Performance,9,10,aston-martin
4306,99,2023,Bahrain Grand Prix,Fernando Alonso,Aston Martin AMR23,Mercedes F1 M14 E Performance,5,3,aston-martin


> We can observe that we have 21 unique Teams and 43 unique engines in our data of analysis

In [226]:
print("Number of unique teams:",len(unique_teams))
unique_engines = engine_data['ENGINE'].unique()
print("Number of unique engines:",len(unique_engines))

Number of unique teams: 21
Number of unique engines: 43


## Final Table

> For the final table, we will merge the **data_table** with the **engine_data** in order to have our desire outcome from which we are going to start our EDA

In [235]:
engine_data['YEAR'] = engine_data['YEAR'].astype(str)
data_table['Year'] = data_table['Year'].astype(str)

# We rename the columns for the engine_data so it allings with the data_table 
engine_data_renamed = engine_data.rename(columns={"Team Name": "Team", "YEAR": "Year"})

# Selecting only the required columns from engine_data
columns_to_merge = ["Team", "Year", "CAR", "ENGINE"]

# Merging the dataframes on 'Team' and 'Year'
merged_data = pd.merge(data_table, engine_data_renamed[columns_to_merge], on=["Team", "Year"], how='left')

merged_data.head()

,Team,Year,ST,W,PD,PP,FL,BF,BG,PTS,WC,CAR,ENGINE
0,mercedes,2023,20,0,7,1,5,2,1,382,2nd,Mercedes F1 W14 E Performance,Mercedes F1 M14 E Performance
1,mercedes,2023,20,0,7,1,5,2,1,382,2nd,Mercedes F1 W14 E Performance,Mercedes F1 M14 E Performance
2,mercedes,2023,20,0,7,1,5,2,1,382,2nd,Mercedes F1 W14 E Performance,Mercedes F1 M14 E Performance
3,mercedes,2023,20,0,7,1,5,2,1,382,2nd,Mercedes F1 W14 E Performance,Mercedes F1 M14 E Performance
4,mercedes,2023,20,0,7,1,5,2,1,382,2nd,Mercedes F1 W14 E Performance,Mercedes F1 M14 E Performance


In [237]:
agg_rules = {
    'ST': 'first',
    'W': 'first',
    'PD': 'first',
    'PP': 'first',
    'FL': 'first',
    'BF': 'first',
    'BG': 'first',
    'PTS': 'first',
    'WC': 'first',
    'CAR': 'first',  
    'ENGINE': 'first'}

# We group by Team and Yeaer so we doesn't have many rows with the same data and we use the agg function to set what we want to do with each column

grouped_data = merged_data.groupby(['Team', 'Year']).agg(agg_rules).reset_index() 
grouped_data

,Team,Year,ST,W,PD,PP,FL,BF,BG,PTS,WC,CAR,ENGINE
0,alfa-romeo,2019,21,0,0,0,0,4,5,57,8th,Alfa Romeo C38,Ferrari 64
1,alfa-romeo,2020,17,0,0,0,0,9,8,8,8th,Alfa Romeo C39,Ferrari 65
2,alfa-romeo,2021,22,0,0,0,0,8,7,13,9th,Alfa Romeo C41,Ferrari 065/6
3,alfa-romeo,2022,22,0,0,0,1,5,5,55,6th,Alfa Romeo C42,Ferrari 066/7
4,alfa-romeo,2023,20,0,0,0,1,8,5,16,9th,Alfa Romeo C43,Ferrari 066/10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,williams,2019,21,0,0,0,0,10,14,1,10th,Williams FW42,Mercedes F1 M10 EQ Power+
86,williams,2020,17,0,0,0,0,11,11,0,10th,Williams FW43,Mercedes M11 EQ Performance
87,williams,2021,22,0,1,0,0,2,2,23,8th,Williams FW43B,Mercedes F1 M12 E Performance
88,williams,2022,22,0,0,0,0,9,6,8,10th,Williams FW44,Mercedes F1 M13 E Performance


In [244]:
grouped_data.to_csv("/Users/patricioulloa/UNI/UBC/Economic~DS/project/final_data.csv") # Finally we save the final datafrae
final_data = pd.read_csv("/Users/patricioulloa/UNI/UBC/Economic~DS/project/final_data.csv")
final_data = final_data.drop("Unnamed: 0", axis=1)
final_data

,Team,Year,ST,W,PD,PP,FL,BF,BG,PTS,WC,CAR,ENGINE
0,alfa-romeo,2019,21,0,0,0,0,4,5,57.0,8th,Alfa Romeo C38,Ferrari 64
1,alfa-romeo,2020,17,0,0,0,0,9,8,8.0,8th,Alfa Romeo C39,Ferrari 65
2,alfa-romeo,2021,22,0,0,0,0,8,7,13.0,9th,Alfa Romeo C41,Ferrari 065/6
3,alfa-romeo,2022,22,0,0,0,1,5,5,55.0,6th,Alfa Romeo C42,Ferrari 066/7
4,alfa-romeo,2023,20,0,0,0,1,8,5,16.0,9th,Alfa Romeo C43,Ferrari 066/10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,williams,2019,21,0,0,0,0,10,14,1.0,10th,Williams FW42,Mercedes F1 M10 EQ Power+
86,williams,2020,17,0,0,0,0,11,11,0.0,10th,Williams FW43,Mercedes M11 EQ Performance
87,williams,2021,22,0,1,0,0,2,2,23.0,8th,Williams FW43B,Mercedes F1 M12 E Performance
88,williams,2022,22,0,0,0,0,9,6,8.0,10th,Williams FW44,Mercedes F1 M13 E Performance


> For the next part of the project. Anna and Ansh will work on the EDA